In [82]:
import pandas as pd
import pymongo
from IPython.display import clear_output
from geopy.geocoders import Nominatim
import json
import re
import string
import concurrent.futures
import requests

In [83]:
# Connect to database
connection_string='mongodb://localhost:27017'
client = pymongo.MongoClient(connection_string)
# Define database for storage
db = client.dice_db

In [84]:
# # Drop Duplicates from database
# import json
# def drop_db_duplicates(db_table):
#     # Retrieve data for analysis
#     actual_collection = db_table.find()
#     #get mongoDB data into a dataframe
#     actual_collection_df = pd.DataFrame(list(actual_collection))
#     #exclude unique index and drop duplicates
#     actual_collection_df = actual_collection_df.loc[:, actual_collection_df.columns != '_id']
#     actual_collection_df = actual_collection_df.drop_duplicates()
#     #prepare records to load into database
#     records = json.loads(actual_collection_df.T.to_json()).values()
#     #insert into database unique values
#     db_table.drop()
#     db_table
#     db_table.insert(records)

In [85]:
# drop_db_duplicates(db.jobs_test)

In [91]:
jobs_collection = db.jobs.find()
jobs_df = pd.DataFrame(list(jobs_collection))
jobs_df

,_id,job_title,job_company,job_salary,job_location,job_date,job_description,location_latitude,location_longitude,job_type,...,data scientist,nlp,pyspark,mongodb,ml,keras,rds,tensorflow,mongo,bootstrap
0,5fd6a28bca0e57fbbf2284e3,Business Analytics Director,University of Kentucky,Depends on Experience,"Lexington, KY, USA",2020-12-12,\nThe University of Kentucky’s Institutional R...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5fd6a28bca0e57fbbf2284e4,Associate Software Engineer III,Penske,Depends on Experience,"Reading, PA, USA",2020-12-12,\nSummary Statement:Working with a diverse tea...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5fd6a28bca0e57fbbf2284e5,Developer,Healh Care Logistics,"$70,000 - $85,000","Grove City, OH, USA",2020-12-11,\nTitle: Developer ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5fd6a28bca0e57fbbf2284e6,ServiceNow Solution Architect,Saxon Global Inc.,$open,"Charlotte, NC, USA",2020-12-13,"\nHi, Please go through the job description an...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5fd6a28bca0e57fbbf2284e7,Full Time HRIS Developer,Saxon Global Inc.,Depends On Experience,"Nashville, TN, USA",2020-12-13,"\nLocation: Nashville TN, Memphis TN , Ridgela...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61796,5fd9864090be9d6978d2592b,People Analytics Consultant,"OUTSOURCE Consulting Services, Inc. (OCSI.co)",Depends on Experience,"Tempe, AZ, USA",2020-12-14,\nSeeking a People Analytics Consultant/Talent...,33.4255,-111.94,Contract W2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61797,5fd9864290be9d6978d2592c,Sr. QA Automation Engineer,Nagarro Inc,Depends on Experience,"Raleigh, NC, USA",2020-12-14,\nPosition: Sr. QA Automation EngineerLocation...,35.7804,-78.6391,"Contract W2, Contract Independent, Contract Co...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61798,5fd9864590be9d6978d2592d,"Enterprise Cloud Architect, Remote",Kutir Inc,,,2020-12-14,\nEnterprise Cloud Architect Location - Remote...,,,"Full Time, Contract Corp-To-Corp",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61799,5fd9864790be9d6978d2592e,HP Exstream Designer,Synechron,Depends on Experience,"Addison, TX, USA",2020-12-14,"\nWe (Synechron, Inc) are looking to hire for ...",32.9604,-96.8303,Full Time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
jobs_df.loc[:, jobs_df.columns != '_id'].drop_duplicates().describe()

,KeyWordsMatch[%],tableau,python,excel,data analyst,machine learning,big data,sql,etl,pandas,...,data scientist,nlp,pyspark,mongodb,ml,keras,rds,tensorflow,mongo,bootstrap
count,33391.000000,1880.000000,11697.000000,4219.000000,795.000000,2004.000000,2786.000000,14318.000000,3455.000000,265.000000,...,504.000000,279.000000,360.000000,1494.000000,771.000000,137.000000,802.000000,341.000000,439.000000,1122.000000
mean,0.072857,1.717021,1.458750,1.217113,1.579874,1.884232,1.711055,1.860525,1.881042,1.139623,...,1.541667,1.555556,1.302778,1.232932,1.911803,1.124088,1.165835,1.208211,1.189066,1.156863
std,0.053017,1.826051,0.907125,0.632066,1.147595,1.707395,1.412232,1.552699,1.690269,0.388441,...,0.967797,1.188810,0.782819,1.019219,1.906926,0.372694,0.467340,0.655782,0.504110,0.431159
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.034483,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,0.068966,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,0.103448,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,...,2.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,0.482759,19.000000,15.000000,21.000000,10.000000,18.000000,19.000000,22.000000,17.000000,3.000000,...,6.000000,10.000000,7.000000,15.000000,26.000000,3.000000,5.000000,7.000000,4.000000,4.000000


In [93]:
jobs_df = jobs_df[["job_title", "job_company", "job_salary", "job_location", "job_date", "job_description"]].drop_duplicates()
jobs_df

,job_title,job_company,job_salary,job_location,job_date,job_description
0,Business Analytics Director,University of Kentucky,Depends on Experience,"Lexington, KY, USA",2020-12-12,\nThe University of Kentucky’s Institutional R...
1,Associate Software Engineer III,Penske,Depends on Experience,"Reading, PA, USA",2020-12-12,\nSummary Statement:Working with a diverse tea...
2,Developer,Healh Care Logistics,"$70,000 - $85,000","Grove City, OH, USA",2020-12-11,\nTitle: Developer ...
3,ServiceNow Solution Architect,Saxon Global Inc.,$open,"Charlotte, NC, USA",2020-12-13,"\nHi, Please go through the job description an..."
4,Full Time HRIS Developer,Saxon Global Inc.,Depends On Experience,"Nashville, TN, USA",2020-12-13,"\nLocation: Nashville TN, Memphis TN , Ridgela..."
...,...,...,...,...,...,...
61796,People Analytics Consultant,"OUTSOURCE Consulting Services, Inc. (OCSI.co)",Depends on Experience,"Tempe, AZ, USA",2020-12-14,\nSeeking a People Analytics Consultant/Talent...
61797,Sr. QA Automation Engineer,Nagarro Inc,Depends on Experience,"Raleigh, NC, USA",2020-12-14,\nPosition: Sr. QA Automation EngineerLocation...
61798,"Enterprise Cloud Architect, Remote",Kutir Inc,,,2020-12-14,\nEnterprise Cloud Architect Location - Remote...
61799,HP Exstream Designer,Synechron,Depends on Experience,"Addison, TX, USA",2020-12-14,"\nWe (Synechron, Inc) are looking to hire for ..."


In [96]:
# Drop Duplicates from database
import json
def insert_df(db_table, dataframe):
    #exclude unique index and drop duplicates
    dataframe = dataframe.drop_duplicates()
    #prepare records to load into database
    records = json.loads(dataframe.T.to_json()).values()
    #insert into database unique values
#     db_table.drop()
#     db_table
    db_table.insert(records)
insert_df(db.jobs_test, jobs_df)

C:\Users\favas\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  # This is added back by InteractiveShellApp.init_path()


In [97]:
# count keywords in description
from collections import Counter

def find_words(key_words, text):
    # Clean-up the string
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)

    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\n', ' ', text)
    
    # Count the words of interest in the string
    text = text.split()
    separator = " "
    words_found = {}
    for word in key_words:
        n_gram = len(word.split())
        res = Counter(separator.join(text[idx : idx + n_gram]) for idx in range(len(text) - n_gram + 1))
        if res[word]>0:
            words_found[word] = res[word]
    # add KeyWordsMatch[%]
    words_found["KeyWordsMatch[%]"] = len(words_found)/len(key_words)
    return words_found

In [98]:
db.jobs_test.find().map(find_words("job_description"))

AttributeError: 'Cursor' object has no attribute 'map'

In [81]:
# # update fields in mongodb
# db.jobs.update_many({"job_date":{'$regex':'None'}},{"$set":{"job_date":'2020-12-12'}})
# list(db.jobs.find({"job_date":{'$regex':'None'}}))

[]

In [12]:
key_words = ["data analyst", "data scientist", "excel", "python", "pandas", "matplotlib", "sql", "postgresql", "bootstrap", "nosql", \
    "mongodb", "mongo", "javascript", "tableau", "machine learning", "ml", "scikit learn", "scikit", "keras", "tensorflow", "pyspark", "natural language processing", \
        "nlp", "big data", "etl", "extract transform load", "amazon web services", "aws", "rds"]

In [ ]:
# jobs_df = pd.read_csv("ProcessedData.csv")
# jobs_df = jobs_df.drop("Unnamed: 0", axis=1)

In [ ]:
# Function to split the salary text
def split_salary(string):
    try:
        amount = re.findall('(\$\d+,\d+|\$\d+)', string)
        salary_min = amount[0]
        salary_min = re.findall("\d+,\d+|\d+",salary_min)[0]
        try:
            salary_max = amount[1]#.group(0)
            salary_max = re.findall("\d+,\d+|\d+",salary_max)[0]
        except:
            salary_max = ""
        um = re.findall('hour|year', string)  
        um = "$/"+um[0]
        return (salary_min, salary_max, um)
    except:
        return ("","","")

In [ ]:
# Split salary text into minimum salary, maximum salary and unit of measurement
salary_list = jobs_df["job_salary"].astype(str).values.tolist()
salary_list = map(split_salary, salary_list)
salary_list = [x for x in salary_list]

In [ ]:
# Record splitted salaries into clean dataframe
salary_list_transposed = zip(*salary_list)
salary_list_transposed = [i for i in salary_list_transposed]
jobs_df["salary_min"] = salary_list_transposed[0]
jobs_df["salary_max"] = salary_list_transposed[1]
jobs_df["UM"] = salary_list_transposed[2]
jobs_df

In [ ]:
# # Insert clean data into new MongoDB table
# #db.jobs_clean.drop()
# clean_records = json.loads(jobs_df.T.to_json()).values()
# db.jobs_clean.insert_many(clean_records)